<a href="https://colab.research.google.com/github/09u2h4n/blender_renderer/blob/main/blender_renderer_v03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set Blender Usage
blender_ver (Blender version): It will download blender and set the path.

---

## Render Single Image Usage

* `file_name` (File name to render): It will be "(file_name).blend" format like chess.blend

* `thread_num` (Thread number): Use amount of threads for rendering and other operations [1-1024], 0 for systems processor count.

* `render_engine` (Render engine): Specify the render engine.

* `frame_number` (Frame number): Render frame. frame_number=10 Render 10th frame.

* `cycles_device` (Cycles device): Override the device that is used to render frames.

* `hybrid_rendering` (Hybrid rendering): If it's checked it will try to render using both CPU and GPU but If `cycles_device` selected CPU it can't render hybrid.

---

## Render Animation Usage

* `file_name` (File name to render): It will be "(file_name).blend" format like chess.blend

* `thread_num` (Thread number): Use amount of threads for rendering and other operations [1-1024], 0 for systems processor count.

* `render_engine` (Render engine): Specify the render engine.

* `use_file_s_frame_settings` (Use file's frame settings): It uses file's frame settings. If it's checked you cant use **Frame's setting**.

* `cycles_device` (Cycles device): Override the device that is used to render frames.

* `hybrid_rendering` (Hybrid rendering): If it's checked it will try to render using both CPU and GPU but If `cycles_device` selected CPU it can't render hybrid.

**Frame's setting**

* `start_frame` (Start frame): Beginning of the frame to render.

* `end_frame` (End frame): End of the frame to render.

* `jump_frame` (Jump frame): It sets jumps frame. Extraction of next and
previous frames.

* `fps` (Frame per second): It sets fps.

---


In [ ]:
# @title #Set Blender
from google.colab import drive, runtime
import os
from time import sleep

!rm -rf /content/sample_data
!mkdir -p /content/render

blender_ver = "4.0.2" # @param ["3.6.4","4.0.2"]
use_blender_on_drive = False

blender_urls = {
    "3.6.4":"https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.6/blender-3.6.4-linux-x64.tar.xz",
    "4.0.2":"https://ftp.nluug.nl/pub/graphics/blender/release/Blender4.0/blender-4.0.2-linux-x64.tar.xz"
}

blender_url = blender_urls[blender_ver]

if use_blender_on_drive:
  try:
    drive.mount("/content/g_drive")
  except:
    print("You have to accept that otherwise, app can't reach blender in your driver!\nIf you want to continue to refuse you should uncheck 'use_on_drive'!")
    sleep(0.9)
    runtime.unassign()
  os.environ["LD_LIBRARY_PATH"] = "/content/g_drive/MyDrive/blender_software_for_colab/blender-3.6.4-linux-x64/lib"
  g_drive_blender_software_path = "/content/g_drive/MyDrive/blender_software_for_colab"
  !mkdir {g_drive_blender_software_path}
  os.chdir(g_drive_blender_software_path)
  list_dir_blender_software = os.listdir()
  if list_dir_blender_software == []:
    list_dir_blender_software.append("debugging")
  blender_dir = f"blender-{blender_ver}-linux-x64"
  for dir in list_dir_blender_software:
    list_dir_blender = os.listdir(blender_dir) if blender_dir in dir else []
    if dir in blender_dir and "blender" in list_dir_blender:
        blender = f"{g_drive_blender_software_path}/{blender_dir}/blender"
        !chmod +x $blender
    else:
      !wget {blender_url} -nc -O /content/blender.tar.xz
      !tar -xf /content/blender.tar.xz
      blender = f"{g_drive_blender_software_path}/{blender_dir}/blender"
      !chmod +x $blender
else:
  !wget {blender_url} -c -O /content/blender.tar.xz
  !tar -xf /content/blender.tar.xz
  blender_dir = f"blender-{blender_ver}-linux-x64"
  blender = f"{blender_dir}/blender"
  !chmod +x $blender

!rm -rf /content/*tar.xz
os.makedirs("/content/textures", exist_ok=True)
os.makedirs("/content/Scripts", exist_ok=True)

In [ ]:
# @title #Texture setter file

%%file /content/Scripts/textures.py
import bpy
import os

# Directory containing the new texture files
new_texture_dir = "/content/textures"  # Make sure to use the correct directory path

# Dictionary to keep track of processed textures
processed_textures = {}

# Iterate through all objects in the scene
for obj in bpy.data.objects:
    # Process only mesh objects
    if obj.type == 'MESH':
        # Iterate through all material slots of the object
        for slot in obj.material_slots:
            if slot.material:
                # Get the node tree of the material
                tree = slot.material.node_tree
                # Iterate through all nodes in the tree
                for node in tree.nodes:
                    # Check if the node is an image texture node and it has an image
                    if node.type == 'TEX_IMAGE' and node.image:
                        # Get the filename of the current image texture
                        filename = os.path.basename(node.image.filepath)
                        # Construct the full path of the new texture file
                        new_texture_path = os.path.join(new_texture_dir, filename)
                        # Update the filepath of the image texture node
                        node.image.filepath = new_texture_path

# Access the world settings
world = bpy.context.scene.world
node = world.node_tree.nodes["Environment Texture"]

# Set the HDRI image path
filename = os.path.basename(node.image.filepath)
new_hdri_path = os.path.join(new_texture_dir, filename)
node.image.filepath = new_hdri_path

# Optionally, you can update the image to reflect the changes
node.image.reload()

In [ ]:
# @title ## Render Single Image

file_name = "file_name_with_blend.blend" # @param {type:"string"}
thread_num = 0 # @param {type:"slider", min:0, max:1024, step:1}
output_ext = "JPEG" # @param ["TGA", "RAWTGA", "JPEG", "IRIS", "IRIZ", "AVIRAW", "AVIJPEG", "PNG", "BMP", "DEFAULT"]
output_ext = " " if output_ext == "DEFAULT" else f" -F {output_ext} "
render_engine = "CYCLES" # @param ["BLENDER_EEVEE", "BLENDER_WORKBENCH", "CYCLES"]
frame_number = 1 # @param {type:"integer"}
frame_number = abs(frame_number)
cycles_device = "CUDA" # @param ["CPU", "CUDA", "OPTIX", "HIP", "ONEAPI", "METAL"]

!{blender} -b /content/render/{file_name} -P /content/Scripts/textures.py -o /content/output/ -t {thread_num}{output_ext}-E {render_engine} -f {frame_number} -- --cycles-device {cycles_device}

---

In [ ]:
# @title ## Render Animation

file_name = "file_name_with_blend.blend" # @param {type:"string"}
thread_num = 0 # @param {type:"slider", min:0, max:1024, step:1}
output_ext = "DEFAULT" # @param ["TGA", "RAWTGA", "JPEG", "IRIS", "IRIZ", "AVIRAW", "AVIJPEG", "PNG", "BMP", "DEFAULT"]
output_ext = " " if output_ext == "DEFAULT" else f" -F {output_ext} "
render_engine = "CYCLES" # @param ["BLENDER_EEVEE", "BLENDER_WORKBENCH", "CYCLES"]
cycles_device = "CUDA" # @param ["CPU", "CUDA", "OPTIX", "HIP", "ONEAPI", "METAL"]

# @markdown ---
# @markdown ## Frame's settings
# @markdown If use_file_s_frame_settings checked you can't use frame setting.
use_file_s_frame_settings = True # @param {type:"boolean"}
start_frame = 0 # @param {type:"integer"}
end_frame = 255 # @param {type:"integer"}
jump_frame = 0 # @param {type:"integer"}
fps = 30 # @param {type:"integer"}
start_frame = abs(start_frame)
end_frame = abs(end_frame)
jump_frame = abs(jump_frame)
fps = abs(fps)
# @markdown ---

if use_file_s_frame_settings:
  !{blender} -b /content/render/{file_name} -P /content/Scripts/textures.py -o /content/output/ -t {thread_num}{output_ext}-E {render_engine} -a -- --cycles-device {cycles_device}
else:
  !{blender} -b /content/render/{file_name} -P /content/Scripts/textures.py -o /content/output/ -t {thread_num}{output_ext}-E {render_engine} -s {start_frame} -e {end_frame} -j {jump_frame} -a -- --cycles-device {cycles_device}

In [6]:
# @title Zip and Download "output" directory a file

import shutil
from google.colab import files

shutil.make_archive("/content/zipped/output", 'zip', "/content/output")

files.download("/content/zipped/output.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>